In [0]:
# Combined Healthcare Claims & Capitation Data Generator
# Generates both claims and capitation data for the same patients to simulate realistic provider financials
# - Jason Votaw 2025

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker

def generate_combined_healthcare_data(clinics, providers_per_clinic, patients_per_provider, months_back=12):
    """
    Generate both capitation and claims data for the same patient population
    """
    fake = Faker()
    Faker.seed(42)  # For reproducible results
    random.seed(42)
    
    capitation_data = []
    claims_data = []
    
    # Generate date range (monthly records)
    today = datetime.today()
    first_of_month = today.replace(day=1)
    date_range = [first_of_month - timedelta(days=30*m) for m in range(months_back)][::-1]
    
    # Provider names for each clinic
    provider_names = [
        'Dr. Sarah Johnson', 'Dr. Michael Chen', 'Dr. Jennifer Williams', 'Dr. David Thompson', 'Dr. Lisa Anderson', 'Dr. Robert Garcia',
        'Dr. Amanda Taylor', 'Dr. Christopher Lee', 'Dr. Michelle Davis', 'Dr. Kevin Brown', 'Dr. Rachel Martinez', 'Dr. Steven Wilson',
        'Dr. Nicole Rodriguez', 'Dr. Daniel Moore', 'Dr. Ashley Clark', 'Dr. Matthew Lewis', 'Dr. Jessica Walker', 'Dr. Andrew Hall',
        'Dr. Emily Young', 'Dr. Brandon King', 'Dr. Samantha Wright', 'Dr. Tyler Lopez', 'Dr. Megan Hill', 'Dr. Jonathan Green'
    ]
    
    # Diagnosis-procedure pairs for claims
    diagnosis_procedure_pairs = {
        'Z00.00': ['99213', '99214', '99396', '99397', '99401'],
        'Z12.31': ['77067', '77066'],
        'Z23': ['90471', '90715', '90686', '90732'],
        'Z12.11': ['45378', '45380'],
        'Z01.411': ['92004', '92012'],
        'I10': ['99213', '99214', '93000', '80053', '36415'],
        'I25.10': ['99214', '93000', '93307', '80053', '36415'],
        'I48.91': ['99214', '93000', '93307', '80053'],
        'I50.9': ['99214', '93000', '93307', '80053', '36415'],
        'R06.02': ['99213', '99214', '71020', '94010'],
        'E11.9': ['99213', '99214', '80053', '83036', '36415'],
        'E78.5': ['99213', '80061', '36415'],
        'E03.9': ['99213', '84443', '36415'],
        'E66.9': ['99213', '99214', '99401'],
        'F32.9': ['99213', '99214', '90834', '90837'],
        'F41.9': ['99213', '99214', '90834', '90837'],
        'F43.10': ['90834', '90837', '99214'],
        'F90.9': ['90834', '90837', '99214'],
        'M54.5': ['99213', '99214', '73060', '97110', '97140'],
        'M25.512': ['99213', '99214', '73030', '20610'],
        'M79.3': ['99213', '99214', '73060', '97110'],
        'M17.11': ['99213', '99214', '73060', '20610'],
        'M75.30': ['99213', '99214', '73030', '20610'],
        'J06.9': ['99213', '99214', '87804', '94010'],
        'J44.1': ['99213', '99214', '94010', '71020'],
        'J45.9': ['99213', '99214', '94010', '94060'],
        'J02.9': ['99213', '87081'],
        'J30.9': ['99213', '99214'],
        'K21.9': ['99213', '99214', '43235'],
        'K59.00': ['99213', '99214', '45378'],
        'K30': ['99213', '99214'],
        'K92.2': ['99214', '43235', '45378'],
        'N39.0': ['99213', '99214', '81001', '87086'],
        'N18.6': ['99214', '80053', '36415'],
        'N40.1': ['99213', '99214', '81001'],
        'N95.1': ['99213', '99214'],
        'L70.0': ['99213', '17000'],
        'L30.9': ['99213', '99214'],
        'L85.9': ['99213', '17000'],
        'C44.92': ['99213', '11100', '17000'],
        'H52.4': ['92004', '92012'],
        'H25.9': ['92004', '66984'],
        'H40.9': ['92004', '92012'],
        'H35.30': ['92004', '92134'],
        'S72.001A': ['99285', '73502', '27236'],
        'I46.9': ['99291', '92950'],
        'R57.0': ['99285', '99291'],
        'J44.0': ['99284', '94760'],
        'N17.9': ['99285', '90935']
    }
    
    # Health plans with product codes and copay structure
    health_plan_products = {
        'Universal Health': {
            'UH-GOLD': {'copay': 20, 'weight': 60},
            'UH-PLATINUM': {'copay': 50, 'weight': 40}
        },
        'BlueDot Health': {
            'BD-BASIC': {'copay': 20, 'weight': 70},
            'BD-PREMIUM': {'copay': 50, 'weight': 30}
        },
        'Medicare': {
            'MEDICARE-A': {'copay': 20, 'weight': 80},
            'MEDICARE-ADVANTAGE': {'copay': 50, 'weight': 20}
        },
        'Medicaid': {
            'MEDICAID-STANDARD': {'copay': 20, 'weight': 100}
        },
        'Self Pay': {
            'SELF-PAY': {'copay': 0, 'weight': 100}
        }
    }
    
    # Place of service and modifier codes
    place_of_service_codes = {
        '11': 'Office',
        '22': 'On Campus-Outpatient Hospital',
        '23': 'Emergency Room - Hospital',
        '21': 'Inpatient Hospital',
        '49': 'Independent Clinic',
        '81': 'Independent Laboratory',
        '24': 'Ambulatory Surgical Center',
        '25': 'Birthing Center',
        '31': 'Skilled Nursing Facility',
        '12': 'Home',
        '02': 'Telehealth',
        '19': 'Off Campus-Outpatient Hospital',
        '32': 'Nursing Facility',
        '71': 'Public Health Clinic',
        '72': 'Rural Health Clinic'
    }
    
    modifier_codes = {
        '25': 'Significant, separately identifiable evaluation and management service',
        '59': 'Distinct procedural service',
        'GT': 'Via synchronous telecommunications system',
        'TC': 'Technical component',
        '26': 'Professional component',
        'LT': 'Left side',
        'RT': 'Right side',
        '50': 'Bilateral procedure',
        'XS': 'Separate structure',
        'XE': 'Separate encounter',
        'XP': 'Separate practitioner',
        'XU': 'Unusual non-overlapping service',
        '95': 'Synchronous telemedicine service',
        'GQ': 'Via asynchronous telecommunications system',
        'GG': 'Performance and payment of a screening mammography'
    }
    
    patient_id = 1
    claim_id = 1
    
    # Master patient registry to ensure consistency
    all_patients = []
    
    for clinic_idx, clinic in enumerate(clinics):
        # Assign providers to clinic
        clinic_providers = provider_names[clinic_idx * providers_per_clinic:(clinic_idx + 1) * providers_per_clinic]
        
        for provider in clinic_providers:
            # Generate base patient list for this provider
            provider_patients = []
            
            for patient_num in range(patients_per_provider):
                # Generate patient demographics using Faker
                sex = random.choice(['M', 'F'])
                age = random.randint(18, 85)
                
                # Small chance for self-pay (2% of patients)
                is_self_pay = random.random() < 0.02
                health_plan = 'Self Pay' if is_self_pay else random.choice(list(health_plan_products.keys())[:-1])
                
                # Select product code based on health plan
                available_products = health_plan_products[health_plan]
                product_codes = list(available_products.keys())
                weights = [available_products[prod]['weight'] for prod in product_codes]
                selected_product = random.choices(product_codes, weights=weights)[0]
                copay_amount = available_products[selected_product]['copay']
                
                # Generate realistic name based on sex
                if sex == 'M':
                    patient_name = fake.name_male()
                else:
                    patient_name = fake.name_female()
                
                patient_id_str = f'PAT{patient_id:06d}'
                member_id = fake.uuid4()[:12].upper()
                
                # Generate baseline risk score (RAF) - higher for older patients
                base_raf = 0.8 + (age - 18) * 0.02
                base_raf += random.uniform(-0.3, 0.7)
                base_raf = max(0.5, min(base_raf, 3.5))
                
                # Calculate base capitation amount based on health plan
                if health_plan == 'Universal Health':
                    plan_multiplier = 1.1
                elif health_plan == 'Self Pay':
                    plan_multiplier = 0
                else:
                    plan_multiplier = 1.0
                
                # Base rates by age group and sex (only if not self-pay)
                if plan_multiplier > 0:
                    if age < 30:
                        base_cap = (180 if sex == 'F' else 150) * plan_multiplier
                    elif age < 50:
                        base_cap = (220 if sex == 'F' else 190) * plan_multiplier
                    elif age < 65:
                        base_cap = (280 if sex == 'F' else 250) * plan_multiplier
                    else:
                        base_cap = (350 if sex == 'F' else 320) * plan_multiplier
                else:
                    base_cap = 0
                
                patient_record = {
                    'patient_id_str': patient_id_str,
                    'patient_name': patient_name,
                    'member_id': member_id,
                    'age': age,
                    'sex': sex,
                    'health_plan': health_plan,
                    'product_code': selected_product,
                    'copay_amount': copay_amount,
                    'clinic': clinic,
                    'provider': provider,
                    'base_raf': round(base_raf, 2),
                    'base_cap': round(base_cap, 2)
                }
                
                provider_patients.append(patient_record)
                all_patients.append(patient_record)
                patient_id += 1
            
            # Generate monthly capitation records with slight patient panel variations
            for month_idx, month_date in enumerate(date_range):
                # Determine how many patients are active this month (±3% variation)
                panel_variation = random.uniform(0.97, 1.03)
                active_patients_count = int(len(provider_patients) * panel_variation)
                active_patients_count = min(active_patients_count, len(provider_patients))
                
                # Randomly select which patients are active this month
                active_patients = random.sample(provider_patients, active_patients_count)
                
                for patient in active_patients:
                    # Only generate capitation for non-self-pay patients
                    if patient['health_plan'] != 'Self Pay':
                        # RAF can vary slightly month to month (±5%)
                        monthly_raf_variation = random.uniform(0.95, 1.05)
                        monthly_raf = round(patient['base_raf'] * monthly_raf_variation, 3)
                        
                        # Capitation amount = base_cap * RAF score
                        capitation_amount = round(patient['base_cap'] * monthly_raf, 2)
                        
                        capitation_data.append({
                            'PatientID': patient['patient_id_str'],
                            'PatientName': patient['patient_name'],
                            'MemberID': patient['member_id'],
                            'Age': patient['age'],
                            'Sex': patient['sex'],
                            'HealthPlan': patient['health_plan'],
                            'ProductCode': patient['product_code'],
                            'Clinic': patient['clinic'],
                            'Provider': patient['provider'],
                            'MonthYear': month_date.strftime('%Y-%m'),
                            'RiskScore_RAF': monthly_raf,
                            'CapitationAmount': capitation_amount
                        })
    
    # Generate claims data for the same patients
    for patient in all_patients:
        # Generate 1-8 claims per patient over the year
        num_claims = random.randint(1, 8)
        
        for claim_num in range(num_claims):
            # Generate claim date within the year
            days_back = random.randint(0, 365)
            claim_date = datetime.now() - timedelta(days=days_back)
            
            # Select matched diagnosis and procedure pair
            selected_dx = random.choice(list(diagnosis_procedure_pairs.keys()))
            available_procedures = diagnosis_procedure_pairs[selected_dx]
            selected_procedure = random.choice(available_procedures)
            
            # For self-pay patients, restrict to office-based care only
            if patient['health_plan'] == 'Self Pay':
                office_procedures = ['99213', '99214', '99203', '99204', '99396', '99397', '99401', '90834', '90837', '81001', '87081']
                office_based_dx = [dx for dx in diagnosis_procedure_pairs.keys() if any(proc in office_procedures for proc in diagnosis_procedure_pairs[dx])]
                selected_dx = random.choice(office_based_dx)
                office_procs_for_dx = [proc for proc in diagnosis_procedure_pairs[selected_dx] if proc in office_procedures]
                if office_procs_for_dx:
                    selected_procedure = random.choice(office_procs_for_dx)
                else:
                    selected_procedure = '99213'
            
            # Select place of service (restrict self-pay to office only)
            if patient['health_plan'] == 'Self Pay':
                place_of_service = '11'
            else:
                place_of_service = random.choice(list(place_of_service_codes.keys()))
            
            # Generate modifiers (0-3 modifiers per claim)
            num_modifiers = random.randint(0, 3)
            selected_modifiers = random.sample(list(modifier_codes.keys()), num_modifiers) if num_modifiers > 0 else []
            
            modifier_1 = selected_modifiers[0] if len(selected_modifiers) > 0 else None
            modifier_2 = selected_modifiers[1] if len(selected_modifiers) > 1 else None
            modifier_3 = selected_modifiers[2] if len(selected_modifiers) > 2 else None
            
            # Generate financial data
            units = random.randint(1, 3)
            charge_amount = round(random.uniform(50, 1500), 2)
            allowed_amount = round(charge_amount * random.uniform(0.7, 0.95), 2)
            
            # Determine claim status
            if patient['health_plan'] == 'Self Pay':
                claim_status = random.choices(['Paid', 'Denied', 'Pending'], weights=[85, 5, 10])[0]
            else:
                claim_status = random.choices(['Paid', 'Denied', 'Pending', 'Partially Paid'], weights=[70, 15, 10, 5])[0]
            
            # Calculate payments based on claim status and health plan
            if patient['health_plan'] == 'Self Pay':
                if claim_status == 'Paid':
                    paid_amount = charge_amount
                    patient_responsibility = 0
                elif claim_status == 'Denied':
                    paid_amount = 0
                    patient_responsibility = charge_amount
                else:
                    paid_amount = 0
                    patient_responsibility = 0
            else:
                if claim_status in ['Paid', 'Partially Paid']:
                    insurance_payment = max(0, allowed_amount - patient['copay_amount'])
                    paid_amount = round(insurance_payment, 2)
                    patient_responsibility = round(patient['copay_amount'] + max(0, charge_amount - allowed_amount), 2)
                elif claim_status == 'Denied':
                    paid_amount = 0
                    patient_responsibility = 0
                else:
                    paid_amount = 0
                    patient_responsibility = 0
            
            claims_data.append({
                'ClaimID': f'CLM{claim_id:08d}',
                'PatientID': patient['patient_id_str'],
                'PatientName': patient['patient_name'],
                'MemberID': patient['member_id'],
                'Age': patient['age'],
                'Sex': patient['sex'],
                'HealthPlan': patient['health_plan'],
                'ProductCode': patient['product_code'],
                'Copay': patient['copay_amount'],
                'Clinic': patient['clinic'],
                'Provider': patient['provider'],
                'ClaimDate': claim_date.strftime('%Y-%m-%d'),
                'PlaceOfService': place_of_service,
                'PlaceOfServiceDesc': place_of_service_codes[place_of_service],
                'PrimaryDiagnosis': selected_dx,
                'PrimaryProcedure': selected_procedure,
                'Modifier1': modifier_1,
                'Modifier2': modifier_2,
                'Modifier3': modifier_3,
                'Units': units,
                'ChargeAmount': charge_amount,
                'AllowedAmount': allowed_amount,
                'PaidAmount': paid_amount,
                'PatientResponsibility': patient_responsibility,
                'ClaimStatus': claim_status
            })
            
            claim_id += 1
    
    return pd.DataFrame(capitation_data), pd.DataFrame(claims_data), pd.DataFrame(all_patients)

def generate_financial_summary(cap_df, claims_df, patients_df):
    """Generate comprehensive financial summary for provider analysis"""
    
    # Revenue analysis
    total_capitation = cap_df['CapitationAmount'].sum()
    total_claims_charges = claims_df['ChargeAmount'].sum()
    total_claims_paid = claims_df['PaidAmount'].sum()
    total_patient_responsibility = claims_df['PatientResponsibility'].sum()
    
    # Provider-level analysis
    provider_revenue = pd.DataFrame()
    
    # Capitation by provider
    cap_by_provider = cap_df.groupby(['Clinic', 'Provider']).agg({
        'CapitationAmount': 'sum',
        'PatientID': 'nunique'
    }).rename(columns={'PatientID': 'CapitatedPatients'})
    
    # Claims by provider
    claims_by_provider = claims_df.groupby(['Clinic', 'Provider']).agg({
        'ChargeAmount': 'sum',
        'PaidAmount': 'sum',
        'PatientResponsibility': 'sum',
        'ClaimID': 'count'
    }).rename(columns={'ClaimID': 'TotalClaims'})
    
    # Combine provider metrics
    provider_summary = cap_by_provider.join(claims_by_provider, how='outer').fillna(0)
    provider_summary['TotalRevenue'] = provider_summary['CapitationAmount'] + provider_summary['PaidAmount']
    provider_summary['RevenuePerPatient'] = provider_summary['TotalRevenue'] / provider_summary['CapitatedPatients']
    
    summary_stats = {
        'total_patients': patients_df.shape[0],
        'total_capitation_revenue': total_capitation,
        'total_claims_charges': total_claims_charges,
        'total_claims_paid': total_claims_paid,
        'total_patient_responsibility': total_patient_responsibility,
        'total_revenue': total_capitation + total_claims_paid,
        'capitation_records': len(cap_df),
        'claims_records': len(claims_df),
        'provider_summary': provider_summary
    }
    
    return summary_stats

# Configuration
clinics = ['North Clinic', 'South Clinic', 'East Clinic', 'West Clinic']
providers_per_clinic = 6
patients_per_provider = 150
months_back = 12

# Generate the combined data
print("Generating combined healthcare financial data...")
df_capitation, df_claims, df_patients = generate_combined_healthcare_data(
    clinics, providers_per_clinic, patients_per_provider, months_back
)

# Generate comprehensive summary
summary = generate_financial_summary(df_capitation, df_claims, df_patients)

# Save all datasets
df_capitation.to_csv("healthcare_capitation_data.csv", index=False)
df_claims.to_csv("healthcare_claims_data.csv", index=False)
df_patients.to_csv("healthcare_patients_registry.csv", index=False)
summary['provider_summary'].to_csv("provider_financial_summary.csv")

print("\n=== FILES GENERATED ===")
print("✅ healthcare_capitation_data.csv")
print("✅ healthcare_claims_data.csv")
print("✅ healthcare_patients_registry.csv")
print("✅ provider_financial_summary.csv")

print(f"\n=== FINANCIAL SUMMARY ===")
print(f"Total unique patients: {summary['total_patients']:,}")
print(f"Capitation records: {summary['capitation_records']:,}")
print(f"Claims records: {summary['claims_records']:,}")
print(f"\n💰 REVENUE BREAKDOWN:")
print(f"Capitation revenue: ${summary['total_capitation_revenue']:,.2f}")
print(f"Claims revenue (paid): ${summary['total_claims_paid']:,.2f}")
print(f"Patient responsibility: ${summary['total_patient_responsibility']:,.2f}")
print(f"TOTAL PROVIDER REVENUE: ${summary['total_revenue']:,.2f}")
print(f"\n📊 CLAIMS ANALYSIS:")
print(f"Total charges submitted: ${summary['total_claims_charges']:,.2f}")
print(f"Collection rate: {(summary['total_claims_paid'] / summary['total_claims_charges'] * 100):.1f}%")
print(f"Revenue mix: {(summary['total_capitation_revenue'] / summary['total_revenue'] * 100):.1f}% Capitation, {(summary['total_claims_paid'] / summary['total_revenue'] * 100):.1f}% Fee-for-Service")

print(f"\n🏥 TOP 5 PROVIDERS BY REVENUE:")
top_providers = summary['provider_summary'].nlargest(5, 'TotalRevenue')
for (clinic, provider), row in top_providers.iterrows():
    print(f"{provider} ({clinic}): ${row['TotalRevenue']:,.2f}")

print(f"\n📋 SAMPLE CAPITATION RECORDS:")
print(df_capitation[['PatientID', 'PatientName', 'HealthPlan', 'Provider', 'MonthYear', 'RiskScore_RAF', 'CapitationAmount']].head().to_string(index=False))

print(f"\n🏥 SAMPLE CLAIMS RECORDS:")
print(df_claims[['ClaimID', 'PatientID', 'PatientName', 'Provider', 'ClaimDate', 'PrimaryDiagnosis', 'ChargeAmount', 'PaidAmount', 'ClaimStatus']].head().to_string(index=False))

Generating combined healthcare financial data...

=== FILES GENERATED ===
✅ healthcare_capitation_data.csv
✅ healthcare_claims_data.csv
✅ healthcare_patients_registry.csv
✅ provider_financial_summary.csv

=== FINANCIAL SUMMARY ===
Total unique patients: 3,600
Capitation records: 41,900
Claims records: 16,331

💰 REVENUE BREAKDOWN:
Capitation revenue: $19,297,528.66
Claims revenue (paid): $7,550,895.27
Patient responsibility: $1,966,413.82
TOTAL PROVIDER REVENUE: $26,848,423.93

📊 CLAIMS ANALYSIS:
Total charges submitted: $12,564,735.97
Collection rate: 60.1%
Revenue mix: 71.9% Capitation, 28.1% Fee-for-Service

🏥 TOP 5 PROVIDERS BY REVENUE:
Dr. Rachel Martinez (South Clinic): $1,200,912.86
Dr. Ashley Clark (East Clinic): $1,195,482.65
Dr. Nicole Rodriguez (East Clinic): $1,191,732.34
Dr. Christopher Lee (South Clinic): $1,185,531.70
Dr. Kevin Brown (South Clinic): $1,177,843.34

📋 SAMPLE CAPITATION RECORDS:
PatientID     PatientName       HealthPlan          Provider MonthYear  RiskScor